## YouTube Ranking

In [38]:
import requests, time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

- Step 1, 2

In [39]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=1'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [40]:
trs = soup.select('tr.aos-init')
len(trs)

0

In [41]:
driver = webdriver.Chrome()
driver.get(url)

In [42]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('tr.aos-init')
len(trs)

100

- Step 3

In [43]:
tr = trs[0]
rank = int(tr.select_one('.rank').get_text().strip())
rank

1

In [44]:
category = tr.select_one('.category').get_text().strip()[1:-1]  # '[음악/댄스/가수]'
category

'음악/댄스/가수'

In [45]:
channel = tr.select_one('.subject > h1 > a').get_text().strip()
channel

'BLACKPINK'

In [46]:
subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
subscriber

'9130만'

In [47]:
view = tr.select_one('.view_cnt').get_text().strip()
video = tr.select_one('.video_cnt').get_text().strip()
view,video

('332억0683만', '567개')

In [48]:
# 숫자로 바꿔주는 함수
def convert(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return int(s)

In [49]:
convert('567개')

567

- Step 4

In [50]:
data = []
for tr in trs:
    rank = int(tr.select_one('.rank').get_text().strip())
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject > h1 > a').get_text().strip()
    subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
    view = convert(tr.select_one('.view_cnt').get_text().strip())
    video = convert(tr.select_one('.video_cnt').get_text().strip())
    data.append({'순위':rank, '채널명':channel, '카테고리':category,
                 '구독자수':subscriber, '조회수':'view', '비디오수':video})

- Step 5. 에러수정

In [51]:
try:
    for index, tr in enumerate(trs):
        rank = int(tr.select_one('.rank').get_text().strip())
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject > h1 > a').get_text().strip()
        subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view = convert(tr.select_one('.view_cnt').get_text().strip())
        video = convert(tr.select_one('.video_cnt').get_text().strip())
except:
    print(index + 1)        # 순위가 출력되게

In [52]:
data = []
for tr in trs:
    rank = int(tr.select_one('.rank').get_text().strip())
    category = tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('.subject > h1 > a').get_text().strip()
    subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
    view = convert(tr.select_one('.view_cnt').get_text().strip())
    video = convert(tr.select_one('.video_cnt').get_text().strip())
    data.append({'순위':rank, '채널명':channel, '카테고리':category,
                 '구독자수':subscriber, '조회수':'view', '비디오수':video})

- Step 6

In [53]:
data = []
for page in range(1,11):
    url = f'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page={page}'
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('tr.aos-init')

    for tr in trs:
        rank = int(tr.select_one('.rank').get_text().strip())
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject > h1 > a').get_text().strip()
        subscriber = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view = convert(tr.select_one('.view_cnt').get_text().strip())
        video = convert(tr.select_one('.video_cnt').get_text().strip())
        data.append({'순위':rank, '채널명':channel, '카테고리':category,
                    '구독자수':subscriber, '조회수':view, '비디오수':video})
        
driver.close()

- Step 7

In [54]:
df = pd.DataFrame(data)
df.head()

,순위,채널명,카테고리,구독자수,조회수,비디오수
0,1,BLACKPINK,음악/댄스/가수,91300000,33206830000,567
1,2,BANGTANTV,음악/댄스/가수,76400000,21348920000,2330
2,3,HYBE LABELS,음악/댄스/가수,72700000,29979900000,1428
3,4,SMTOWN,음악/댄스/가수,32100000,27856680000,4227
4,5,JYP Entertainment,음악/댄스/가수,27800000,20510660000,1833


In [55]:
df.to_csv('data/유튜브랭킹_20230919.csv', index=False)
pd.read_csv('data/유튜브랭킹_20230919.csv').tail()

,순위,채널명,카테고리,구독자수,조회수,비디오수
995,997,성창경TV,미분류,770000,1196450000,18314
996,996,빅페이스 BIGFACE,미분류,770000,183120000,142
997,998,꾸삐KUPI,키즈/어린이,770000,570580000,986
998,1000,서은이야기 [V-log],키즈/어린이,770000,298990000,628
999,999,LDF TV by lottedutyfree,TV/방송,770000,656800000,1517
